In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [3]:
# Barra covariance matrix

barra = pd.read_csv("grad_cov_matrix_aug_26_2024.csv", index_col=0)

barra = barra.reset_index()
barra = barra.drop(columns=['Barrid'])

barra

,AAPL,C,CMG,COKE,DRI,GEF,KLAC,LULU,MELI,MP,NFLX,RMD,RSG,_ETF29
0,0.066252,0.019871,0.022422,0.009795,0.015394,0.017353,0.057025,0.030330,0.020060,0.041054,0.021479,0.019979,0.011409,0.023602
1,0.019871,0.095950,0.026281,0.010310,0.033818,0.045105,0.052686,0.048360,0.019102,0.074358,0.023693,0.024621,0.016636,0.029849
2,0.022422,0.026281,0.087754,0.016615,0.038984,0.021162,0.046708,0.048354,0.027369,0.058526,0.023850,0.030193,0.013336,0.025367
3,0.009795,0.010310,0.016615,0.100737,0.013366,0.010280,0.017829,0.013257,0.014649,0.015419,0.012132,0.015540,0.010231,0.012367
4,0.015394,0.033818,0.038984,0.013366,0.064022,0.028712,0.035145,0.046026,0.021083,0.056850,0.017655,0.026325,0.014606,0.021920
5,0.017353,0.045105,0.021162,0.010280,0.028712,0.080132,0.040121,0.044762,0.015985,0.063917,0.016789,0.022915,0.015962,0.023505
6,0.057025,0.052686,0.046708,0.017829,0.035145,0.040121,0.178829,0.069473,0.040260,0.109368,0.041298,0.044975,0.016569,0.050129
7,0.030330,0.048360,0.048354,0.013257,0.046026,0.044762,0.069473,0.186583,0.039893,0.108228,0.032400,0.043294,0.015515,0.036744
8,0.020060,0.019102,0.027369,0.014649,0.021083,0.015985,0.040260,0.039893,0.128931,0.043788,0.021359,0.023156,0.011822,0.021831
9,0.041054,0.074358,0.058526,0.015419,0.056850,0.063917,0.109368,0.108228,0.043788,0.336423,0.041296,0.060265,0.018780,0.052609


In [5]:
# Get covariance matrix

cov_matrix = barra.to_numpy()

cov_matrix

In [6]:
# Data Preparation

# Parameters
tickers = barra.columns.to_list()

start = '2016-01-01'

# Rename _ETF29 to IWV
download_tickers = tickers.copy()
download_tickers[-1] = 'IWV'

# YF download
df = yf.download(download_tickers, start=start)

# Reformat Dataframe
df = df.stack().reset_index()

# Add daily return column
df['Return'] = df.groupby("Ticker")['Adj Close'].pct_change()

df = df.dropna()


# Rename IWV to _ETF29
df['Ticker'] = np.where(df['Ticker'] == 'IWV', '_ETF29', df['Ticker'])

df

[*********************100%%**********************]  14 of 14 completed
C:\Users\andre\AppData\Local\Temp\ipykernel_13620\201692936.py:16: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack().reset_index()


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Return
13,2016-01-05,AAPL,23.288244,25.677500,26.462500,25.602501,26.437500,223164000.0,-0.025059
14,2016-01-05,C,39.391636,50.860001,51.610001,50.410000,51.540001,17444900.0,-0.005280
15,2016-01-05,CMG,8.980600,8.980600,9.197600,8.936000,9.000000,108065000.0,0.000490
16,2016-01-05,COKE,166.498169,176.740005,179.410004,174.300003,177.020004,32600.0,0.009654
17,2016-01-05,DRI,49.820759,63.889999,63.889999,62.709999,63.490002,2077500.0,0.018167
...,...,...,...,...,...,...,...,...,...
29349,2024-08-27,MELI,2030.130005,2030.130005,2048.989990,2002.989990,2002.989990,189211.0,0.012685
29350,2024-08-27,MP,12.760000,12.760000,12.910000,12.650000,12.780000,931695.0,-0.010853
29351,2024-08-27,NFLX,696.755005,696.755005,707.794983,686.919983,688.530029,2496215.0,0.012078
29352,2024-08-27,RMD,240.820007,240.820007,241.000000,226.820007,226.139999,1275365.0,0.066236


In [7]:
# Returns dataframe
return_matrix = df[['Date','Ticker','Return']].pivot(index='Date', columns='Ticker', values='Return').dropna()

return_matrix

Ticker,AAPL,C,CMG,COKE,DRI,GEF,KLAC,LULU,MELI,MP,NFLX,RMD,RSG,_ETF29
Date,,,,,,,,,,,,,,
2020-06-23,0.021345,0.017096,-0.029049,-0.004938,0.056061,-0.002404,0.007036,-0.011586,-0.007161,-0.009000,-0.003803,0.015856,0.001231,0.003976
2020-06-24,-0.017652,-0.040415,0.008459,-0.014160,-0.060252,-0.036758,-0.023756,-0.019724,-0.024266,-0.013118,-0.018037,-0.000495,-0.023976,-0.026623
2020-06-25,0.013276,0.036804,0.000907,0.006726,0.053382,0.019081,0.019775,-0.000305,0.018526,0.012270,0.017604,0.024414,0.009574,0.011358
2020-06-26,-0.030726,-0.058846,-0.015016,-0.046209,-0.021987,-0.009208,-0.014453,-0.018975,0.001781,0.004040,-0.048314,-0.003596,0.004742,-0.022853
2020-06-29,0.023047,0.014320,0.012389,0.047634,0.050446,0.044920,0.011394,0.016648,0.007871,0.002918,0.008660,0.014491,0.007079,0.014810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-21,-0.000486,-0.012722,0.034463,0.042052,0.003094,0.015045,0.019185,0.019302,0.000897,0.070674,-0.002033,0.005154,0.002533,0.003715
2024-08-22,-0.008260,-0.001817,-0.005211,0.026262,-0.011116,-0.002094,-0.034601,-0.008207,-0.009280,-0.053111,-0.011705,-0.003968,0.005296,-0.008657
2024-08-23,0.010288,0.028467,0.004864,-0.004417,0.001105,0.029706,0.024195,0.019358,0.005772,0.037660,-0.003237,0.014458,-0.003093,0.013320


In [9]:
# Create assets dataframe

assets = df.groupby('Ticker')['Return'].mean().rename("Expected Return").to_frame().reset_index()

assets['Beta'] = cov_matrix[-1] / cov_matrix[-1,-1]

assets

,Ticker,Expected Return,Beta
0,AAPL,0.001204,0.941837
1,C,0.000430,1.191137
2,CMG,0.001090,1.012263
3,COKE,0.001249,0.493516
4,DRI,0.000800,0.874714
5,GEF,0.000749,0.937971
6,KLAC,0.001518,2.000393
7,LULU,0.001024,1.466281
8,MELI,0.001835,0.871179
9,MP,0.001101,2.099351


In [10]:
# Expected Returns Vector
expected_returns = assets['Expected Return'].values

expected_returns

array([0.00120381, 0.00042988, 0.00109024, 0.00124875, 0.00079984,
       0.00074921, 0.00151839, 0.00102441, 0.00183472, 0.0011008 ,
       0.00122716, 0.00095677, 0.00086318, 0.00058755])

In [11]:
# Beta Vector
betas = assets['Beta'].values

betas

array([0.94183742, 1.19113683, 1.0122631 , 0.49351639, 0.874714  ,
       0.9379715 , 2.00039253, 1.46628139, 0.87117891, 2.0993512 ,
       0.86937809, 0.96709378, 0.51023708, 1.        ])

In [12]:
# Portfolio metric functions
def portfolio_xs_return(weights, expected_returns):
    bmk_return = expected_returns[-1]
    return weights @ expected_returns - bmk_return

def portfolio_beta(weights, betas):
    return weights @ betas

def portfolio_tracking_error(current_weights, cov_matrix):
    active_weights = current_weights.copy()
    active_weights[-1] -= 1
    return np.sqrt(active_weights @ cov_matrix @ active_weights)

def negative_information_ratio(current_weights, expected_returns, cov_matrix):
    port_er = portfolio_xs_return(current_weights, expected_returns)
    port_te = portfolio_tracking_error(current_weights, cov_matrix)
    return -port_er / port_te

In [13]:
# Display Weights Function
def display_weights(tickers, weights):
    df = pd.DataFrame()
    df['Ticker'] = tickers
    df['Weight'] = [round(weight, 2) for weight in weights]

    display(df)

In [14]:
# Baseline results

# Initial Weights
size = len(tickers)
initial_weights = np.array([1/size] * size)

# Baseline Results
port_return = portfolio_xs_return(initial_weights, expected_returns)
port_tracking_error = portfolio_tracking_error(initial_weights, cov_matrix)
port_ir = port_return / port_tracking_error

#Display
display_weights(tickers, initial_weights)
print(f"Portfolio Return: {round(port_return,2)} %")
print(f"Portfolio Tracking Error: {round(port_tracking_error,2)} %")
print(f"Portfolio Information Ratio: {round(port_ir,2)}")
print(f"Weights sum to {round(initial_weights.sum(),2)}")

,Ticker,Weight
0,AAPL,0.07
1,C,0.07
2,CMG,0.07
3,COKE,0.07
4,DRI,0.07
5,GEF,0.07
6,KLAC,0.07
7,LULU,0.07
8,MELI,0.07
9,MP,0.07


Portfolio Return: 0.0 %
Portfolio Tracking Error: 0.08 %
Portfolio Information Ratio: 0.01
Weights sum to 1.0


In [15]:
# Optimization

# Constraints and bounds
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # weights sum to 1
    {'type': 'eq', 'fun': lambda x: portfolio_beta(x, betas) - 1},  # beta of 1
    {'type': 'eq', 'fun': lambda x: portfolio_tracking_error(x, cov_matrix) - .05},  # tracking error of .05
  ]

# Result
result = minimize(negative_information_ratio, initial_weights, 
                  args=(expected_returns, cov_matrix),
                  method='SLSQP', constraints=constraints)

optimal_weights = result.x

optimal_weights

array([0.09032368, 0.00297144, 0.0412963 , 0.05049277, 0.01548881,
       0.03252591, 0.08614201, 0.01108257, 0.07414479, 0.01412462,
       0.05112245, 0.02660833, 0.11445481, 0.38922151])

In [16]:
# Optimal Results

opt_port_tracking_error = portfolio_tracking_error(optimal_weights,cov_matrix)
opt_port_return = portfolio_xs_return(optimal_weights, expected_returns)
opt_port_beta = portfolio_beta(optimal_weights, betas)
opt_port_information_ratio = opt_port_return / opt_port_tracking_error

# Display
display_weights(tickers,optimal_weights)
print(f"Optimized Portfolio Beta: {round(opt_port_beta,2)}")
print(f"Optimized Portfolio Return: {round(opt_port_return,2) } %")
print(f"Optimized Portfolio Tracking Error: {round(opt_port_tracking_error,2)}")
print(f"Optimized Portfolio Information Ratio: {round(opt_port_information_ratio,2)}")
print(f"Weights sum to: {round(optimal_weights.sum(), 2)}")

,Ticker,Weight
0,AAPL,0.09
1,C,0.00
2,CMG,0.04
3,COKE,0.05
4,DRI,0.02
5,GEF,0.03
6,KLAC,0.09
7,LULU,0.01
8,MELI,0.07
9,MP,0.01


Optimized Portfolio Beta: 1.0
Optimized Portfolio Return: 0.0 %
Optimized Portfolio Tracking Error: 0.05
Optimized Portfolio Information Ratio: 0.01
Weights sum to: 1.0
